## Ungraded Lab: Lambda Layer

This lab will show how you can define custom layers with the [Lambda](https://keras.io/api/layers/core_layers/lambda/) layer. You can either use [lambda functions](https://www.w3schools.com/python/python_lambda.asp) within the Lambda layer or define a custom function that the Lambda layer will call. Let's get started!

## Imports

In [13]:
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np

## Prepare the Dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build the Model: with inplace lambda layer

Here, we'll use a Lambda layer to define a custom layer in our network. We're using a lambda function to get the absolute value of the layer input.

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
    
  # Usually, we did: tf.keras.layers.Dense(128, activation='relu'),  
  # But here, we use custom lambda layer
  tf.keras.layers.Dense(128),
  tf.keras.layers.Lambda(lambda x: tf.abs(x)),
    
  tf.keras.layers.Dense(10, activation='softmax')
])

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 618us/step - loss: 0.0299 - accuracy: 0.9905
Epoch 2/5
1875/1875 [==============================] - 1s 614us/step - loss: 0.0206 - accuracy: 0.9942
Epoch 3/5
1875/1875 [==============================] - 1s 611us/step - loss: 0.0174 - accuracy: 0.9948
Epoch 4/5
1875/1875 [==============================] - 1s 613us/step - loss: 0.0183 - accuracy: 0.9940
Epoch 5/5
1875/1875 [==============================] - 1s 616us/step - loss: 0.0142 - accuracy: 0.9951


In [6]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 476us/step - loss: 0.1010 - accuracy: 0.9762


[0.10098610073328018, 0.9761999845504761]

Another way to use the Lambda layer is to pass in a function defined outside the model. The code below shows how a custom ReLU function is used as a custom layer in the model.

## 1. Lambda layer with a function

In [8]:
def my_relu(x):
    return K.maximum(-0.1, x)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    
    # Usually, we did: tf.keras.layers.Dense(128, activation='relu'),  
    # But here, we use custom lambda layer with a defined function
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu), 
    
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 632us/step - loss: 0.4280 - accuracy: 0.8781
Epoch 2/5
1875/1875 [==============================] - 1s 644us/step - loss: 0.1214 - accuracy: 0.9649
Epoch 3/5
1875/1875 [==============================] - 1s 655us/step - loss: 0.0775 - accuracy: 0.9772
Epoch 4/5
1875/1875 [==============================] - 1s 626us/step - loss: 0.0540 - accuracy: 0.9838
Epoch 5/5
1875/1875 [==============================] - 1s 651us/step - loss: 0.0426 - accuracy: 0.9873


In [9]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 470us/step - loss: 0.0754 - accuracy: 0.9766


[0.07535514235496521, 0.9765999913215637]

## 2. Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [10]:
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):
    ''' Inherit from this Layer class of Tensorflow '''

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights) and biases'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

Now we can use our custom layer like below:

In [11]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

print(f'x = {x}')
print(f'y = {y}')

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

x = [[1.]]
y = [[-0.02502577]]
[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.02502577]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Let's then try using it in simple network:

In [14]:
# define the dataset: y = 2x - 1
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

# perform inference
print(model.predict([10.0]))  #Output y_pred = [[18.981586]]. Note x = 10.0 --> y = 2x-1=19

# see the updated state of the variables
print(my_layer.variables)

[[18.981586]]
[<tf.Variable 'simple_dense_1/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973315]], dtype=float32)>, <tf.Variable 'simple_dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-0.991727], dtype=float32)>]


## 3. Custom Layer with weights and adding activation layer  
To use the built-in activations in Keras, we can specify an `activation` parameter in the `__init__()` method of our custom layer class. From there, we can initialize it by using the `tf.keras.activations.get()` method. This takes in a string identifier that corresponds to one of the [available activations](https://keras.io/api/layers/activations/#available-activations) in Keras. Next, you can now pass in the forward computation to this activation in the `call()` method.

In [15]:
class SimpleDense(Layer):

    # add an activation parameter
    def __init__(self, units=32, activation=None):
        super(SimpleDense, self).__init__()
        self.units = units
        
        # define the activation to get from the built-in activation layers in Keras
        self.activation = tf.keras.activations.get(activation)


    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)


    def call(self, inputs):
        
        # pass the computation to the activation layer
        return self.activation(tf.matmul(inputs, self.w) + self.b)

We can now pass in an activation parameter to our custom layer. The string identifier is mostly the same as the function name so 'relu' below will get `tf.keras.activations.relu`.

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    
    # Usually we did: tf.keras.layer.Dense(128, activation='relu')
    # But below, we use our own Simple Dense Layer with activation
    SimpleDense(128, activation='relu'),
        
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 696us/step - loss: 0.4805 - accuracy: 0.8597
Epoch 2/5
1875/1875 [==============================] - 1s 682us/step - loss: 0.1489 - accuracy: 0.9547
Epoch 3/5
1875/1875 [==============================] - 1s 669us/step - loss: 0.1117 - accuracy: 0.9665
Epoch 4/5
1875/1875 [==============================] - 1s 669us/step - loss: 0.0858 - accuracy: 0.9742
Epoch 5/5
1875/1875 [==============================] - 1s 701us/step - loss: 0.0735 - accuracy: 0.9771


In [17]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 492us/step - loss: 0.0734 - accuracy: 0.9764


[0.07340724766254425, 0.9764000177383423]